## Linear Regression

# imports
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split



In [47]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.linear_model import Ridge

pd.set_option("display.max_columns", None)

In [19]:
# Read the Dataframe here
#load data
jan_2019 = pd.read_csv('../Kyle_eda/data/01_2019_flights_weather.csv', low_memory=False)
# drop rows with nan in arr_delay
jan_2019.dropna(inplace=True,subset=['arr_delay'])

In [23]:
# encode mkt_unique_carrier
enc = OneHotEncoder()
enc.fit(jan_2019['mkt_unique_carrier'].values.reshape(-1,1))
enc_df = pd.DataFrame(enc.transform(jan_2019['mkt_unique_carrier'].values.reshape(-1,1)).toarray())

enc_df.reset_index(drop=True, inplace=True)
jan_2019.reset_index(drop=True, inplace=True)
jan_2019 = pd.concat([jan_2019, enc_df], axis=1,)

In [24]:
# get mapping dictionary from file to use for encoding origin and dest
dest_dict = pd.read_csv('../Kyle_eda/data/dest_dict.csv', index_col=0)
origin_dict = pd.read_csv('../Kyle_eda/data/origin_dict.csv', index_col=0)
dest_dict = dest_dict.T.to_dict('records')[0]
origin_dict = origin_dict.T.to_dict('records')[0]

In [27]:
# encode origin and dest
jan_2019['dest'] = jan_2019['dest'].map(dest_dict)
jan_2019['origin'] = jan_2019['origin'].map(origin_dict)
jan_2019

,date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,cancellation_code,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name,crs_dep_time_hr,crs_arr_time_hr,time_origin,description_origin,precip_origin,windspeedKmph_origin,winddirDegree_origin,visibility_origin,DewPointC_origin,pressure_origin,cloudcover_origin,WindGustKmph_origin,humidity_origin,tempC_origin,city_origin,time_hr_origin,time_dest,description_dest,precip_dest,windspeedKmph_dest,winddirDegree_dest,visibility_dest,DewPointC_dest,pressure_dest,cloudcover_dest,WindGustKmph_dest,humidity_dest,tempC_dest,city_dest,time_hr_dest,0,1,2,3,4,5,6,7,8,9,0,1,2,3,4,5,6,7,8,9,0,1,2,3,4,5,6,7,8,9
0,2019-01-01,AA,AA_CODESHARE,AA,5621,OH,N720PS,5621,11057,7.062739,"Charlotte, NC",13360,2.238713,"Melbourne, FL",2019-01-01 11:40:00,1139.0,-1.0,30.0,1209.0,1325.0,3.0,2019-01-01 13:28:00,1328.0,0.0,0.0,NaN,0.0,N,108.0,109.0,76.0,1.0,490.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,13,2019-01-01 12:00:00,Sunny,0.0,10,245,10,15,1018,14,19,72,21,CLT,12,2019-01-01 13:00:00,Partly cloudy,0.0,13,172,10,19,1021,38,8,68,25,MLB,13,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-01-01,AA,AA,AA,1950,AA,N831AW,1950,11057,7.062739,"Charlotte, NC",14027,6.482724,"West Palm Beach/Palm Beach, FL",2019-01-01 11:31:00,1124.0,-7.0,14.0,1138.0,1314.0,4.0,2019-01-01 13:25:00,1318.0,-7.0,0.0,NaN,0.0,N,114.0,114.0,96.0,1.0,590.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,13,2019-01-01 12:00:00,Sunny,0.0,10,245,10,15,1018,14,19,72,21,CLT,12,2019-01-01 13:00:00,Light rain shower,0.0,17,138,10,21,1021,56,21,76,25,PBI,13,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-01-01,DL,DL,DL,354,DL,N976DL,354,10397,2.697050,"Atlanta, GA",14027,6.482724,"West Palm Beach/Palm Beach, FL",2019-01-01 11:24:00,1121.0,-3.0,12.0,1133.0,1259.0,5.0,2019-01-01 13:12:00,1304.0,-8.0,0.0,NaN,0.0,N,108.0,103.0,86.0,1.0,545.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,13,2019-01-01 11:00:00,Patchy rain possible,0.2,11,312,9,15,1020,57,10,88,17,ATL,11,2019-01-01 13:00:00,Light rain shower,0.0,17,138,10,21,1021,56,21,76,25,PBI,13,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-01-01,DL,DL,DL,898,DL,N319US,898,12953,6.358496,"New York, NY",14027,6.482724,"West Palm Beach/Palm Beach, FL",2019-01-01 10:05:00,1004.0,-1.0,19.0,1023.0,1252.0,4.0,2019-01-01 13:24:00,1256.0,-28.0,0.0,NaN,0.0,N,199.0,172.0,149.0,1.0,1035.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,13,2019-01-01 10:00:00,Sunny,0.0,32,274,10,6,1009,14,45,65,12,LGA,10,2019-01-01 13:00:00,Light rain shower,0.0,17,138,10,21,1021,56,21,76,25,PBI,13,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-01-01,B6,B6,B6,2953,B6,N949JB,2953,12478,2.662853,"New York, NY",14027,6.482724,"West Palm Beach/Palm Beach, FL",2019-01-01 10:17:00,1031.0,14.0,17.0,1048.0,1317.0,5.0,2019-01-01 13:13:00,1322.0,9.0,0.0,NaN,0.0,N,176.0,171.0,149.0,1.0,1028.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,13,2019-01-01 10:00:00,Sunny,0.0,30,273,10,6,1009,14,58,68,12,JFK,10,2019-01-01 13:00:00,Light rain shower,0.0,17,138,10,21,1021,56,21,76,25,PBI,13,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [28]:
# mask for columns to keep
cols_to_keep = ['origin', 
                'dest', 
                'crs_elapsed_time',
                'distance',
                'crs_dep_time_hr',
                'crs_arr_time_hr',
                'precip_origin',
                'windspeedKmph_origin',
                'winddirDegree_origin',
                'visibility_origin',
                'DewPointC_origin',
                'pressure_origin',
                'cloudcover_origin',
                'WindGustKmph_origin',
                'humidity_origin',
                'tempC_origin',
                'precip_dest',
                'windspeedKmph_dest',
                'winddirDegree_dest',
                'visibility_dest',
                'DewPointC_dest',
                'pressure_dest',
                'cloudcover_dest',
                'WindGustKmph_dest',
                'humidity_dest',
                'tempC_dest']

X = jan_2019[cols_to_keep]
y = jan_2019['arr_delay']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [32]:
# Standard Scaler creation using the fit_transform() method
scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

In [45]:
scaled_X_train.shape

(432354, 26)

In [52]:
# Fitting the model
LR_model = LinearRegression()
LR_model.fit(scaled_X_train,y_train)

LinearRegression()

In [55]:
y_pred = LR_model.predict(scaled_X_test)

In [56]:
# The coefficients
print('Coefficients: \n', LR_model.coef_)

# The mean squared error
print('Mean squared error: %.2f \n'
      % metrics.mean_squared_error(y_test, y_pred))

# The coefficient of determination: 
print('Coefficient of determination: %.2f \n'
      % metrics.r2_score(y_test, y_pred))

Coefficients: 
 [  2.66695497   4.39653868  -5.23249981   4.59910654   2.47904177
   1.68017412   1.86110586   2.30471858  -0.40078238  -3.12297042
 -13.05827681   0.16107745  -0.16695811   0.21758754   6.89205761
  10.12313455   2.06546187   1.69293602   0.16631948  -1.5792258
  -8.49659198  -0.40730895   0.1753039    1.0062904    5.61381081
   6.66171662]
Mean squared error: 2676.00 

Coefficient of determination: 0.05 



### Implementing Ridge

In [48]:
# Ridge regression
Rid_regression = Ridge()

In [57]:
# fit the data
Rid_regression.fit(scaled_X_train, y_train)

Ridge()

In [58]:
ridg_y_pred = Rid_regression.predict(scaled_X_test)

In [60]:
# The coefficients
print('Coefficients: \n', Rid_regression.coef_)

# The mean squared error
print('Mean squared error: %.2f'
      % metrics.mean_squared_error(y_test, ridg_y_pred))


# The coefficient of determination: 
print('Coefficient of determination: %.2f'
      % metrics.r2_score(y_test, ridg_y_pred))

Coefficients: 
 [  2.66691863   4.39652187  -5.23199736   4.59859264   2.47903032
   1.68016775   1.86112949   2.3046462   -0.40079428  -3.12306026
 -13.05450045   0.16106124  -0.16691327   0.21755864   6.89014974
  10.11959042   2.06547004   1.69289126   0.16632565  -1.57928007
  -8.49444854  -0.40730619   0.17532255   1.00630178   5.61274079
   6.65968489]
Mean squared error: 2676.00
Coefficient of determination: 0.05
